### 기본 세팅
작동 시간이 너무 빠르면 자동툴로 인식하는 경우가 있어 계속 sleep을 걸음

In [187]:
import os
import time
import pandas as pd

from selenium import webdriver
import pyperclip
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup 

In [67]:
# 셀레니움 드라이버로 웹 실행
driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(3)

In [68]:
# 클립보드에 input을 복사하고, ActionChain을 이용해 복사 내용을 로그인 폼에 붙여넣기
def copy_input(xpath, input):
    pyperclip.copy(input)
    driver.find_element_by_xpath(xpath).click()
    ActionChains(driver).key_down(Keys.CONTROL).send_keys('v').key_up(Keys.CONTROL).perform()
    time.sleep(1)

In [69]:
# 네이버 id & pw 입력
naver_id = 'bokju128'
naver_pw = 'uks01uks28!'

In [70]:
# 네이버 로그인 
driver.get('https://nid.naver.com/nidlogin.login')             
time.sleep(1)

copy_input('//*[@id="id"]', naver_id)
time.sleep(1)
copy_input('//*[@id="pw"]', naver_pw)
time.sleep(1)

driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

### 주소 및 닉네임 크롤링 테스트
INFP & ENFP 게시판: menuid=18

In [159]:
page = 0
address = []
nickname = []

while page <= 1:
    try:
        # 페이지 및 iframe 전환
        page += 1
        driver.get(f"https://cafe.naver.com/ArticleList.nhn?search.clubid=11856775&search.menuid=18&search.boardtype=L&search.page={page}")
        driver.switch_to.frame("cafe_main")
        
        # 더이상 게시글이 없을 경우
        try:
            if driver.find_element_by_class_name("nodata").text == '등록된 게시글이 없습니다.':
                break
        except:
            pass
        
        # 페이지당 주소 및 닉네임 20개 크롤링
        for i in range(1, 31, 1):
            address.append(driver.find_element_by_xpath(f"/html/body/div[1]/div/div[4]/table/tbody/tr[{i}]/td[1]/div[2]/div/a[1]").get_attribute("href"))
            nickname.append(driver.find_element_by_xpath(f"/html/body/div[1]/div/div[4]/table/tbody/tr[{i}]/td[2]/div/table/tbody/tr/td/a").get_attribute("onclick"))
                            
        # 기존 웹페이지로 복귀
        driver.switch_to.default_content()   
        
    except:
        pass

In [160]:
# 크롤링한 게시글 주소
address

['https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504221&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504217&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504173&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504161&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504102&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504086&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype=L&articleid=504047&referrerAllArticles=false',
 'https://cafe.naver.com/ArticleRead.nhn?clubid=11856775&page=1&menuid=18&boardtype

In [161]:
len(address)

30

In [162]:
# 크롤링한 닉네임이 포함되어 있는 onclick 타입
nickname

["ui(event, 'pumpkim9',3,'isfj 여','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'pumpkim9',3,'isfj 여','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'koe1012',3,'INFP흰여우','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, '0tweety0',3,'요나ESTP','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'heesong04',3,'국화빵ENFP','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'rkdms100101',3,'EsFxj','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'dkzkt123',3,'끌림intp','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'yey_',3,'조각INFP','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'qpalzmwoaj',3,'지구젤리','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return f

In [163]:
len(nickname)

30

In [164]:
nickname[0]

"ui(event, 'pumpkim9',3,'isfj 여','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;"

In [165]:
type(nickname[0])

str

### onclick 타입 nickname에서 mbti 추출

In [166]:
mbti_group = ['intj', 'intp', 'entj', 'entp',
              'infj', 'infp', 'enfj', 'enfp',
              'istj', 'isfj', 'estj', 'esfj',
              'istp', 'isfp', 'estp', 'esfp']
len(mbti_group)

16

In [167]:
# onclick 타입 nickname 소문자화
for i in range(len(nickname)):
    nickname[i] = nickname[i].lower()

In [168]:
drop_idx = []   

for i in range(len(nickname)):
    check_cnt = 0
    
    for mbti in mbti_group:        
        # 닉네임에 mbti가 포함된 경우 닉네임을 mbti화
        if mbti in nickname[i]:
            nickname[i] = mbti
        
        # 닉네임에 mbti가 포함되지 않은 경우
        if mbti not in nickname[i]:
            check_cnt += 1
        
    # 닉네임에 mbti가 아예 없을 경우 해당 인덱스 저장
    if check_cnt == 16:
        drop_idx.append(i)

In [169]:
nickname

['isfj',
 'isfj',
 'infp',
 'estp',
 'enfp',
 "ui(event, 'rkdms100101',3,'esfxj','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'intp',
 'infp',
 "ui(event, 'qpalzmwoaj',3,'지구젤리','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'enfj',
 "ui(event, 'uji990709',3,'뭘까','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'enfp',
 'enfp',
 'infp',
 'infp',
 'infp',
 "ui(event, 'feel7381',3,'경이xstp','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'infj',
 'infj',
 "ui(event, 'nalshow',3,'꿀꾸리미녀','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 "ui(event, 'summer_0810',3,'인쁘피와 인티제','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'infp',
 'enfp',
 'infp',
 'enfp',
 'enfj',
 'istp',
 'enfp',
 "ui(event, 'kksh1005',3,'픕히','11856775','me', 'false', 'true', 'mbticafe', 'false', '18'); return false;",
 'intj']

In [170]:
drop_idx   # 닉네임에 mbti가 포함되지 않아 drop할 인덱스

[5, 8, 10, 16, 19, 20, 28]

In [171]:
len(drop_idx)

7

### 기존 address, nickname에서 drop_idx에 해당되면 드랍

In [172]:
print(' drop전 address 개수: {0}'.format( len(address) ))
print(' drop전 nickname 개수: {0}'.format( len(nickname) ))

 drop전 address 개수: 30
 drop전 nickname 개수: 30


In [173]:
for i in range(len(address)):
    
    while i in drop_idx:
        address.pop(i)
        nickname.pop(i)
        drop_idx.remove(i)
        
        for idx in range(len(drop_idx)):
            drop_idx[idx] -= 1
    
print(' drop후 address 개수: {0}'.format( len(address) ))
print(' drop후 nickname 개수: {0}'.format( len(nickname) ))

 drop후 address 개수: 23
 drop후 nickname 개수: 23


In [184]:
nickname

['isfj',
 'isfj',
 'infp',
 'estp',
 'enfp',
 'intp',
 'infp',
 'enfj',
 'enfp',
 'enfp',
 'infp',
 'infp',
 'infp',
 'infj',
 'infj',
 'infp',
 'enfp',
 'infp',
 'enfp',
 'enfj',
 'istp',
 'enfp',
 'intj']

### address, nickname으로 게시글 제목과 본문 추출

In [196]:
title = []
content = []

for url in address:
    # 페이지 및 iframe 전환
    driver.get(url)
    driver.switch_to.frame("cafe_main")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제목과 본문 크롤링
    cur_title = driver.find_element_by_css_selector("h3.title_text").text
    cur_content = soup.select("#tbody")[0].select('p')

    title.append(cur_title)
    content.append(cur_content)

IndexError: list index out of range

In [194]:
# 크롤링한 게시글 제목
title

['ENFP분들 적극적인? 여자 어떠시나요',
 'ENFP와 연락',
 '.',
 '제가 원래 마음의 문을 잘 안 여는데',
 '예전에 좋아했던 ENFP',
 '직설적인거 싫어한다며.. 상처 받는다며..',
 '중2포토 만들어드려용',
 '엔픕 사람 잘 못믿나요???ENFP 봐주세용',
 '주변 사람에게 미완의 작품을 보여 줬을 때 얻는 위험',
 '이상형발견 급해요 어떡하죠',
 '인프피를 가장 괴롭게 하는 법',
 '남 뒷담을 즐겨 하는 사람들의 심리가 뭘까요?',
 '노래 하나 추천하고 갑니다',
 '엔ㅍ프피 친구의 상처주는 말',
 'ENFP 분들은 어떤 타입을 좋아하시고 또 어떤 타입을 싫어하시나요?',
 '3~5년전 이맘때..',
 '엔프피 갬성 블로그',
 'ESFJ와 연애해보신 INFP 있으신가요?',
 '엔프피는 어느유형이랑 친구많이할까요?',
 '엔프피랑 연애중인 엔프제입니다',
 '엔프피남에 대한 고찰',
 '감정의 발작(부정적인 말 주의)?',
 '엔프피 원래 이렇게 따뜻해요??']

In [195]:
# 크롤링한 게시글 본문
content

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

In [ ]:
#data_crawled_from_naver_cafe = pd.DataFrame({'mbti': nickname, 'title': title, 'content': content})